In [1]:
import pandas as pd
import numpy as np
from xgboost.sklearn import XGBClassifier
import xgboost as xgb
from matplotlib import pyplot as plt

%matplotlib inline

In [2]:
test = pd.read_csv('../data/raw/X_test.csv')
x_train = pd.read_csv('../data/raw/X_train.csv')
y_train = pd.read_csv('../data/raw/y_train.csv')
sample = pd.read_csv('../data/raw/sample_submission.csv')

In [3]:
x_train.head()

,row_id,series_id,measurement_number,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z
0,0_0,0,0,-0.75853,-0.63435,-0.10488,-0.10597,0.107650,0.017561,0.000767,-0.74857,2.1030,-9.7532
1,0_1,0,1,-0.75853,-0.63434,-0.10490,-0.10600,0.067851,0.029939,0.003385,0.33995,1.5064,-9.4128
2,0_2,0,2,-0.75853,-0.63435,-0.10492,-0.10597,0.007275,0.028934,-0.005978,-0.26429,1.5922,-8.7267
3,0_3,0,3,-0.75852,-0.63436,-0.10495,-0.10597,-0.013053,0.019448,-0.008974,0.42684,1.0993,-10.0960
4,0_4,0,4,-0.75852,-0.63435,-0.10495,-0.10596,0.005135,0.007652,0.005245,-0.50969,1.4689,-10.4410


In [4]:
y_train.head()

,series_id,group_id,surface
0,0,13,fine_concrete
1,1,31,concrete
2,2,20,concrete
3,3,31,concrete
4,4,22,soft_tiles


In [15]:
x_train.groupby(['series_id']).agg(['min','max','mean','median'])

measurement_number                   orientation_X            \
                         min  max  mean median           min       max   
series_id                                                                
0                          0  127  63.5   63.5     -0.759530 -0.758220   
1                          0  127  63.5   63.5     -0.958960 -0.958370   
2                          0  127  63.5   63.5     -0.514340 -0.509440   
3                          0  127  63.5   63.5     -0.939680 -0.938840   
4                          0  127  63.5   63.5     -0.896890 -0.886730   
5                          0  127  63.5   63.5      0.464030  0.465450   
6                          0  127  63.5   63.5     -0.405750 -0.398560   
7                          0  127  63.5   63.5     -0.926190 -0.925590   
8                          0  127  63.5   63.5      0.009159  0.018242   
9                          0  127  63.5   63.5      0.751010  0.824630   
10                         0  127  63.5   63.5      0.847110  0.850250   
11                         0  127  63.5   63.5      0.487410  0.502940   
12                         0  127  63.5   63.5      0.329340  0.338450   
13                         0  127  63.5   63.5      0.367210  0.380330   
14                         0  127  63.5   63.5     -0.505490 -0.502700   
15                         0  127  63.5   63.5     -0.943070 -0.941630   
16                         0  127  63.5   63.5     -0.961380 -0.957860   
17                         0  127  63.5   63.5      0.434790  0.443610   
18                         0  127  63.5   63.5     -0.295970 -0.282130   
19                         0  127  63.5   63.5      0.821490  0.836870   
20                         0  127  63.5   63.5      0.908650  0.916300   
21                         0  127  63.5   63.5      0.013220  0.013481   
22                         0  127  63.5   63.5     -0.962170 -0.961930   
23                         0  127  63.5   63.5     -0.285430 -0.281010   
24                         0  127  63.5   63.5      0.952620  0.954960   
25                         0  127  63.5   63.5      0.337360  0.355030   
26                         0  127  63.5   63.5     -0.826040 -0.812610   
27                         0  127  63.5   63.5      0.621400  0.652160   
28                         0  127  63.5   63.5      0.939840  0.940780   
29                         0  127  63.5   63.5      0.012365  0.014542   
...                      ...  ...   ...    ...           ...       ...   
3780                       0  127  63.5   63.5     -0.495500 -0.489580   
3781                       0  127  63.5   63.5      0.563990  0.587760   
3782                       0  127  63.5   63.5      0.459200  0.460170   
3783                       0  127  63.5   63.5      0.475140  0.477880   
3784                       0  127  63.5   63.5     -0.985510 -0.984790   
3785                       0  127  63.5   63.5     -0.789090 -0.778810   
3786                       0  127  63.5   63.5     -0.884730 -0.882370   
3787                       0  127  63.5   63.5     -0.835900 -0.833310   
3788                       0  127  63.5   63.5      0.659660  0.667310   
3789                       0  127  63.5   63.5     -0.614600 -0.540820   
3790                       0  127  63.5   63.5      0.709350  0.711420   
3791                       0  127  63.5   63.5      0.802750  0.806110   
3792                       0  127  63.5   63.5      0.638470  0.642620   
3793                       0  127  63.5   63.5     -0.978180 -0.975690   
3794                       0  127  63.5   63.5      0.755410  0.774660   
3795                       0  127  63.5   63.5     -0.353110 -0.345300   
3796                       0  127  63.5   63.5     -0.771010 -0.746130   
3797                       0  127  63.5   63.5     -0.165460 -0.060392   
3798                       0  127  63.5   63.5     -0.487330 -0.484990   
3799                       0  127  63.5   63.5     -0.179230 -0.169810   
3800          